In [1]:
# Install necessary libraries
!pip install transformers datasets torch

import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import mean_squared_error

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Load the training and test datasets from GitHub
train_url = 'https://raw.githubusercontent.com/Kushal-Chandani/Deep-Learning-Project/main/public_data/train/track_b/eng.csv'
test_url = 'https://raw.githubusercontent.com/Kushal-Chandani/Deep-Learning-Project/main/public_data/dev/track_b/eng_b.csv'

train_df = pd.read_csv(train_url)
test_df = pd.read_csv(test_url)

In [3]:
# Define emotion columns
emotions = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Preprocess function for tokenizing and preparing labels
def preprocess_data(texts, labels):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)

    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels.tolist()  # Ensure labels are float
    }

# Split and preprocess the training and evaluation data
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    train_df['text'],
    train_df[emotions].values.astype(float),  # Ensure labels are float for regression
    test_size=0.2,
    random_state=42
)

test_texts = test_df['text']

# train_encodings = preprocess_data(train_texts, train_labels)
# eval_encodings = preprocess_data(eval_texts, eval_labels)

# # Convert to Huggingface dataset format
# train_dataset = Dataset.from_dict(train_encodings)
# eval_dataset = Dataset.from_dict(eval_encodings)

In [4]:
import torch
import torch.nn as nn
from transformers import DebertaV2Tokenizer, DebertaV2Model, DebertaV2PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import os

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Define a custom model for regression with DeBERTa
class DebertaForMultiLabelRegression(DebertaV2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.deberta = DebertaV2Model(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, self.num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.deberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)  # Regression logits

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()  # Using MSE loss for regression
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Load tokenizer and model
deberta_tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')
deberta_model = DebertaForMultiLabelRegression.from_pretrained('microsoft/deberta-v3-base', num_labels=len(emotions))

# Tokenization and preprocessing
def preprocess_data(texts, labels):
    encodings = deberta_tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels, dtype=torch.float)
    }

# Prepare datasets
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)
train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)

# Training arguments and trainer setup
training_args = TrainingArguments(
    output_dir='./deberta_results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./deberta_logs',
    logging_steps=10,
    save_strategy="epoch"
)

# Define Trainer
deberta_trainer = Trainer(
    model=deberta_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: {
        "mse": ((p.predictions - p.label_ids) ** 2).mean().item()
    }
)

# Train the model
deberta_trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaForMultiLabelRegression were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=695, training_loss=0.25450300878758053, metrics={'train_runtime': 490.2814, 'train_samples_per_second': 22.579, 'train_steps_per_second': 1.418, 'total_flos': 677973231122400.0, 'train_loss': 0.25450300878758053, 'epoch': 5.0})

In [24]:
import torch
import torch.nn as nn
from transformers import DebertaV2Tokenizer, DebertaV2Model, DebertaV2PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import os

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Define a custom model for regression with DeBERTa
class DebertaForMultiLabelRegression(DebertaV2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.deberta = DebertaV2Model(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, self.num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.deberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)  # Regression logits

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()  # Using MSE loss for regression
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Load tokenizer and model
debertaL_tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-large')
debertaL_model = DebertaForMultiLabelRegression.from_pretrained('microsoft/deberta-v3-large', num_labels=len(emotions))

# Tokenization and preprocessing
def preprocess_data(texts, labels):
    encodings = debertaL_tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels, dtype=torch.float)
    }

# Prepare datasets
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)
train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)

# Training arguments and trainer setup
training_args = TrainingArguments(
    output_dir='./deberta_results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./deberta_logs',
    logging_steps=10,
    save_strategy="epoch"
)

# Define Trainer
debertaL_trainer = Trainer(
    model=deberta_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: {
        "mse": ((p.predictions - p.label_ids) ** 2).mean().item()
    }
)

# Train the model
debertaL_trainer.train()


Some weights of DebertaForMultiLabelRegression were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=695, training_loss=0.05797149781271708, metrics={'train_runtime': 717.6685, 'train_samples_per_second': 15.425, 'train_steps_per_second': 0.968, 'total_flos': 677973231122400.0, 'train_loss': 0.05797149781271708, 'epoch': 5.0})

In [7]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
import os
from transformers import TrainingArguments

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Define a custom model for regression
class BertForMultiLabelRegression(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs[1]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)  # Regression logits

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()  # Using MSE loss for regression
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Load the tokenizer and custom regression model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMultiLabelRegression.from_pretrained('bert-base-uncased', num_labels=len(emotions))

# Tokenization and preprocessing
def preprocess_data(texts, labels):
    encodings = bert_tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels, dtype=torch.float)   # Convert to tensor of floats for regression
    }

# Preprocess the training and evaluation data
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)

# Convert the preprocessed data into Hugging Face Dataset format
train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)



# Update TrainingArguments to use `eval_strategy` instead of `evaluation_strategy`
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',  # Use eval_strategy instead of evaluation_strategy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch"
)

# Define the Trainer with MSE loss for regression
bert_trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: {
        "mse": ((p.predictions - p.label_ids) ** 2).mean().item()  # MSE metric
    }
)

# Train the model
bert_trainer.train()


Some weights of BertForMultiLabelRegression were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=695, training_loss=0.18164010871228553, metrics={'train_runtime': 334.5936, 'train_samples_per_second': 33.085, 'train_steps_per_second': 2.077, 'total_flos': 699734949141420.0, 'train_loss': 0.18164010871228553, 'epoch': 5.0})

In [8]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Define a custom model for regression with DistilBERT
class DistilBertForMultiLabelRegression(DistilBertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.distilbert = DistilBertModel(config)
        self.dropout = nn.Dropout(config.seq_classif_dropout)
        self.regressor = nn.Linear(config.dim, self.num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)  # Regression logits

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()  # MSE loss for regression
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Load tokenizer and model
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForMultiLabelRegression.from_pretrained('distilbert-base-uncased', num_labels=len(emotions))

# Tokenization and preprocessing
def preprocess_data(texts, labels):
    encodings = distilbert_tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels, dtype=torch.float)   # Convert to tensor of floats for regression
    }

# Preprocess data
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)
train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)

# Training arguments and trainer setup
training_args = TrainingArguments(
    output_dir='./distilbert_results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./distilbert_logs',
    logging_steps=10,
    save_strategy="epoch"
)

# Define Trainer
distilbert_trainer = Trainer(
    model=distilbert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: {
        "mse": ((p.predictions - p.label_ids) ** 2).mean().item()
    }
)

# Train the model
distilbert_trainer.train()


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForMultiLabelRegression were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=695, training_loss=0.1990540805480463, metrics={'train_runtime': 178.9371, 'train_samples_per_second': 61.865, 'train_steps_per_second': 3.884, 'total_flos': 347476987121580.0, 'train_loss': 0.1990540805480463, 'epoch': 5.0})

In [10]:
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel, RobertaPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
import os
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Disable Weights & Biases logging
os.environ["WANDB_DISABLED"] = "true"

# Define a custom model for regression
class RobertaForMultiLabelRegression(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, self.num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs[1]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)  # Regression logits

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()  # Using MSE loss for regression
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Load the tokenizer and custom regression model
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMultiLabelRegression.from_pretrained('roberta-base', num_labels=len(emotions))

# Tokenization and preprocessing
def preprocess_data(texts, labels):
    encodings = roberta_tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels, dtype=torch.float)   # Convert to tensor of floats for regression
    }

# Preprocess the training and evaluation data
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)

# Convert the preprocessed data into Hugging Face Dataset format
train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_strategy="epoch"
)

# Define the Trainer with MSE loss for regression
roberta_trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: {
        "mse": ((p.predictions - p.label_ids) ** 2).mean().item()  # MSE metric
    }
)

# Train the model
roberta_trainer.train()


Some weights of RobertaForMultiLabelRegression were not initialized from the model checkpoint at roberta-base and are newly initialized: ['regressor.bias', 'regressor.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=695, training_loss=0.21583699802700565, metrics={'train_runtime': 374.243, 'train_samples_per_second': 29.58, 'train_steps_per_second': 1.857, 'total_flos': 620090320783860.0, 'train_loss': 0.21583699802700565, 'epoch': 5.0})

In [13]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 39.4 MB/s eta 0:00:00


In [15]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaPreTrainedModel

# Define a custom model for regression with XLM-Roberta
class XLMRobertaForMultiLabelRegression(XLMRobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = XLMRobertaModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, self.num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token output
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)  # Regression logits

        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()  # Using MSE loss for regression
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

# Load the correct tokenizer and model
xlm_tokenizer = XLMRobertaTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')
xlm_model = XLMRobertaForMultiLabelRegression.from_pretrained('FacebookAI/xlm-roberta-base', num_labels=len(emotions))

# Continue

# Tokenization and preprocessing
def preprocess_data(texts, labels):
    encodings = xlm_tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return {
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': torch.tensor(labels, dtype=torch.float)   # Convert to tensor of floats for regression
    }

# Preprocess data
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)
train_dataset = Dataset.from_dict(train_encodings)
eval_dataset = Dataset.from_dict(eval_encodings)

# Training arguments and trainer setup
training_args = TrainingArguments(
    output_dir='./xlm_results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./xlm_logs',
    logging_steps=10,
    save_strategy="epoch"
)

# Define Trainer
xlm_trainer = Trainer(
    model=xlm_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: {
        "mse": ((p.predictions - p.label_ids) ** 2).mean().item()
    }
)

# Train the model
xlm_trainer.train()


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForMultiLabelRegression were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=695, training_loss=0.44535474794374097, metrics={'train_runtime': 568.3138, 'train_samples_per_second': 19.479, 'train_steps_per_second': 1.223, 'total_flos': 659912634962640.0, 'train_loss': 0.44535474794374097, 'epoch': 5.0})

In [ ]:
# model.save_pretrained('./saved_model')

In [ ]:
# import numpy as np
# import torch
# from tqdm import tqdm  # Import for progress bar

# # Use fine-tuned BERT as a feature extractor with batching and progress bar
# def extract_finetuned_features(texts, batch_size=64):
#     embeddings = []
#     model.eval()  # Set BERT to evaluation mode

#     # Loop over texts in batches
#     for i in tqdm(range(0, len(texts), batch_size), desc="Extracting features"):
#         batch_texts = texts[i:i + batch_size]

#         # Check if batch_texts is non-empty
#         if not batch_texts:
#             continue

#         inputs = tokenizer(batch_texts, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
#         inputs = {k: v.to('cuda') for k, v in inputs.items()}  # Move inputs to GPU

#         with torch.no_grad():
#             outputs = model.bert(**inputs)

#         cls_embeddings = outputs.pooler_output.cpu().numpy()  # Move to CPU and convert to numpy
#         embeddings.extend(cls_embeddings)  # Append batch embeddings

#     return np.array(embeddings)

# # Ensure model and tokenizer are on the GPU
# model.to('cuda')

# # Extract features for train, eval, and test sets
# train_features = extract_finetuned_features(list(train_texts), batch_size=32)
# eval_features = extract_finetuned_features(list(eval_texts), batch_size=32)
# test_features = extract_finetuned_features(list(test_texts), batch_size=32)


In [ ]:
# import numpy as np
# from sklearn.metrics import mean_squared_error
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from sklearn.multioutput import MultiOutputRegressor

# # Assuming train_features, eval_features, test_features are already defined
# # and train_labels, eval_labels are the true labels for training and evaluation.

# # 1. Train XGBoost Model
# xgb_model = MultiOutputRegressor(XGBRegressor(n_estimators=150, max_depth=5, learning_rate=0.2))
# xgb_model.fit(train_features, train_labels)

# # 2. Make predictions with XGBoost
# xgb_predictions = xgb_model.predict(eval_features)

# # 3. Evaluate XGBoost Model
# mse_xgb = mean_squared_error(eval_labels, xgb_predictions)
# print(f"Validation MSE for XGBoost: {mse_xgb}")

# # 4. Train LightGBM Model
# lgb_model = MultiOutputRegressor(LGBMRegressor(n_estimators=150, max_depth=5, learning_rate=0.2))
# lgb_model.fit(train_features, train_labels)

# # 5. Make predictions with LightGBM
# lgb_predictions = lgb_model.predict(eval_features)

# # 6. Evaluate LightGBM Model
# mse_lgb = mean_squared_error(eval_labels, lgb_predictions)
# print(f"Validation MSE for LightGBM: {mse_lgb}")

# # 7. Select the best model based on MSE
# if mse_xgb < mse_lgb:
#     print("Using XGBoost as the best model.")
#     best_model = xgb_model
# else:
#     print("Using LightGBM as the best model.")
#     best_model = lgb_model




In [16]:
def preprocess_test_data(test_texts,tokenizer):
    encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask']
    })

In [ ]:
# def ensemble_predict_classes(texts):
#     # Tokenize for BERT
#     bert_encodings = preprocess_test_data(texts,b_tokenizer)
#     with torch.no_grad():
#         bert_outputs = bert_model(**bert_encodings)
#         bert_logits = bert_outputs.logits

#     # Tokenize for RoBERTa
#     roberta_encodings = preprocess_test_data(texts,r_tokenizer)
#     with torch.no_grad():
#         roberta_outputs = robert_model(**roberta_encodings)
#         roberta_logits = roberta_outputs.logits

#     # Average the logits from both models
#     ensemble_logits = (bert_logits + roberta_logits) / 2

#     # Convert logits to classes in 0-3 scale
#     ensemble_classes = torch.clamp(torch.round(ensemble_logits), min=0, max=3).int()

#     return ensemble_classes.numpy()  # Convert to numpy array for easier handling


# # Assuming `test_texts` is a list of text data for testing and `test_df` contains an 'id' column
# test_predictions = ensemble_predict_classes(test_texts)

# # Prepare the test DataFrame for saving
# for i, emotion in enumerate(emotions):
#     test_df[emotion] = test_predictions[:, i]

# # Save the predictions to CSV
# test_df[['id'] + emotions].to_csv('pred_eng_b.csv', index=False)



In [35]:


# Get predictions from RoBERTa model
roberta_predictions = roberta_trainer.predict(preprocess_test_data(test_df['text'],roberta_tokenizer)).predictions

# Get predictions from BERT model
bert_predictions = bert_trainer.predict(preprocess_test_data(test_df['text'],bert_tokenizer)).predictions

# Get predictions from Deberta model
deberta_predictions = deberta_trainer.predict(preprocess_test_data(test_df['text'],deberta_tokenizer)).predictions

# Get predictions from DebertaL model
debertaL_predictions = debertaL_trainer.predict(preprocess_test_data(test_df['text'],deberta_tokenizer)).predictions

# Get predictions from Deberta model
distilbert_predictions = distilbert_trainer.predict(preprocess_test_data(test_df['text'],distilbert_tokenizer)).predictions

# Get predictions from Deberta model
xlm_predictions = xlm_trainer.predict(preprocess_test_data(test_df['text'],xlm_tokenizer)).predictions

# Perform ensembling by averaging the predictions from both models
ensemble_predictions = ( deberta_predictions + debertaL_predictions + roberta_predictions ) / 3

# Convert ensemble predictions to intensity scores and add them to test dataframe
for i, emotion in enumerate(emotions):
    test_df[emotion] = ensemble_predictions[:, i]

# Display the first few rows of predictions to confirm
print(test_df.head())

# # Save the predictions to a CSV file
# test_df[['id', 'text'] + emotions].to_csv('emotion_intensity_predictions_ensemble.csv', index=False)


                      id                                               text  \
0  eng_dev_track_b_00001  I noticed this months after moving in and doin...   
1  eng_dev_track_b_00002               can't wait to be in another wedding!   
2  eng_dev_track_b_00003  Just getting out of the house put a smile on m...   
3  eng_dev_track_b_00004                    Your sister is a heavy sleeper.   
4  eng_dev_track_b_00005                            The police were called.   

      Anger      Fear       Joy   Sadness  Surprise  
0 -0.083913  0.188739  0.488126  0.068413  0.109830  
1 -0.183055  0.207226  2.438634  0.086399  0.440641  
2  0.136409 -0.137803  1.954952 -0.007457  0.233465  
3  0.054838  0.433414  0.105134  0.288748  0.035105  
4  0.413135  2.197246 -0.132332  0.879031  1.147485  


In [36]:
# Define a function to convert scores to classes
def convert_to_classes(scores):
    # Round the scores and clip to range [0, 3]
    return np.clip(np.round(scores).astype(int), 0, 3)

# Convert predictions to classes for each emotion
for emotion in emotions:
    test_df[emotion] = convert_to_classes(ensemble_predictions[:, emotions.index(emotion)])

# Display the first few rows of predictions
print(test_df.head())


test_df[['id'] + emotions].to_csv('pred_eng_b.csv', index=False)

                      id                                               text  \
0  eng_dev_track_b_00001  I noticed this months after moving in and doin...   
1  eng_dev_track_b_00002               can't wait to be in another wedding!   
2  eng_dev_track_b_00003  Just getting out of the house put a smile on m...   
3  eng_dev_track_b_00004                    Your sister is a heavy sleeper.   
4  eng_dev_track_b_00005                            The police were called.   

   Anger  Fear  Joy  Sadness  Surprise  
0      0     0    0        0         0  
1      0     0    2        0         0  
2      0     0    2        0         0  
3      0     0    0        0         0  
4      0     2    0        1         1  
